In [1]:
data_dir = "/content/drive/MyDrive/training_setwm"
model_checkpoint = "openai/clip-vit-base-patch16"

In [2]:
! pip install datasets transformers accelerate

In [3]:
from datasets import load_dataset
dataset = load_dataset("imagefolder", data_dir=data_dir)

Resolving data files:   0%|          | 0/547 [00:00<?, ?it/s]

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 547
    })
})

In [5]:
dataset["train"][540]

{'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=554x470>,
 'label': 1}

In [6]:
dataset["train"].features

{'image': Image(decode=True, id=None),
 'label': ClassLabel(names=['benign_breast_cancer', 'malignant_breast_cancer'], id=None)}

In [7]:
from transformers import AutoImageProcessor

image_processor  = AutoImageProcessor.from_pretrained(model_checkpoint)

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.


In [8]:
image_processor

CLIPImageProcessor {
  "crop_size": {
    "height": 224,
    "width": 224
  },
  "do_center_crop": true,
  "do_convert_rgb": true,
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.48145466,
    0.4578275,
    0.40821073
  ],
  "image_processor_type": "CLIPImageProcessor",
  "image_std": [
    0.26862954,
    0.26130258,
    0.27577711
  ],
  "resample": 3,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "shortest_edge": 224
  }
}

In [9]:
from torchvision.transforms import (
    CenterCrop,
    Compose,
    Normalize,
    RandomHorizontalFlip,
    RandomResizedCrop,
    Resize,
    ToTensor,
)

normalize = Normalize(mean=image_processor.image_mean, std=image_processor.image_std)
if "height" in image_processor.size:
    size = (image_processor.size["height"], image_processor.size["width"])
    crop_size = size
    max_size = None
elif "shortest_edge" in image_processor.size:
    size = image_processor.size["shortest_edge"]
    crop_size = (size, size)
    max_size = image_processor.size.get("longest_edge")

train_transforms = Compose(
        [
            RandomResizedCrop(crop_size),
            RandomHorizontalFlip(),
            ToTensor(),
            normalize,
        ]
    )

val_transforms = Compose(
        [
            Resize(size),
            CenterCrop(crop_size),
            ToTensor(),
            normalize,
        ]
    )

def preprocess_train(example_batch):
    """Apply train_transforms across a batch."""
    example_batch["pixel_values"] = [
        train_transforms(image.convert("RGB")) for image in example_batch["image"]
    ]
    return example_batch

def preprocess_val(example_batch):
    """Apply val_transforms across a batch."""
    example_batch["pixel_values"] = [val_transforms(image.convert("RGB")) for image in example_batch["image"]]
    return example_batch

In [10]:
# split up training into training + validation
splits = dataset["train"].train_test_split(test_size=0.1)
train_ds = splits['train']
val_ds = splits['test']

In [11]:
train_ds.set_transform(preprocess_train)
val_ds.set_transform(preprocess_val)

In [12]:
train_ds[0]

{'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=460x391>,
 'label': 0,
 'pixel_values': tensor([[[-3.9081e-01, -3.4702e-01, -3.6162e-01,  ...,  7.9166e-01,
            1.0106e+00,  1.0398e+00],
          [-2.7403e-01, -2.4483e-01, -2.5943e-01,  ...,  4.2670e-01,
            5.8728e-01,  5.8728e-01],
          [-2.5943e-01, -2.4483e-01, -2.3023e-01,  ...,  1.0553e-01,
            1.4933e-01,  1.2013e-01],
          ...,
          [-1.0477e+00, -1.0769e+00, -1.0477e+00,  ..., -1.6463e+00,
           -1.6463e+00, -1.6171e+00],
          [-8.2877e-01, -8.4336e-01, -7.9957e-01,  ..., -1.6463e+00,
           -1.6463e+00, -1.6171e+00],
          [-5.5140e-01, -5.0760e-01, -4.9300e-01,  ..., -1.5587e+00,
           -1.5587e+00, -1.5441e+00]],
 
         [[-3.1135e-01, -2.6633e-01, -2.8134e-01,  ...,  9.0428e-01,
            1.1294e+00,  1.1594e+00],
          [-1.9129e-01, -1.6127e-01, -1.7628e-01,  ...,  5.2908e-01,
            6.9417e-01,  6.9417e-01],
          [-1.7628e-01, 

In [13]:
labels = dataset["train"].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = i
    id2label[i] = label

id2label[0]

'benign_breast_cancer'

In [14]:
from transformers import AutoModelForImageClassification, TrainingArguments, Trainer

model = AutoModelForImageClassification.from_pretrained(
    model_checkpoint,
    label2id=label2id,
    id2label=id2label,
    ignore_mismatched_sizes = True, # provide this in case you're planning to fine-tune an already fine-tuned checkpoint
)

Some weights of CLIPForImageClassification were not initialized from the model checkpoint at openai/clip-vit-base-patch16 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
model_name = model_checkpoint.split("/")[-1]

batch_size = 32

args = TrainingArguments(
    f"clip-vit-finetuned-breastcancer",
    remove_unused_columns=False,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=True,
)

In [16]:
from datasets import load_metric

metric = load_metric("accuracy")

<ipython-input-16-350fcae91df9>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [17]:
import numpy as np

# the compute_metrics function takes a Named Tuple as input:
# predictions, which are the logits of the model as Numpy arrays,
# and label_ids, which are the ground-truth labels as Numpy arrays.
def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

In [18]:
import torch

def collate_fn(examples):
    pixel_values = torch.stack([example["pixel_values"] for example in examples])
    labels = torch.tensor([example["label"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}

In [19]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=image_processor,
    compute_metrics=compute_metrics,
    data_collator=collate_fn,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [20]:
trainer.train()
trainer.push_to_hub()

Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.670284,0.690909
2,No log,0.625423,0.690909
3,0.654400,0.618040,0.690909


events.out.tfevents.1711501211.a42a7893934c.9806.0:   0%|          | 0.00/6.15k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/as-cle-bert/clip-vit-finetuned-breastcancer/commit/60d4a7feea77b6b1b26d9401daff4641ec432b68', commit_message='End of training', commit_description='', oid='60d4a7feea77b6b1b26d9401daff4641ec432b68', pr_url=None, pr_revision=None, pr_num=None)